Frequently used scripts

I found out that trying to use GPU can be very tricky. I though my TensorFlow was not utilizing my GPU as setting up dedicated and integrated  graphics cards on a laptop can get rather obscure in Linux, I gained remote access to my brother's computer (with 8GB GeForce GTX 1070) running Windows and thought I would set everything up there (that was pain on its own as the docs for Windows are outdated), I successfully prepared everything and in the mean time found out that my computer indeed was not using the GPU. With a lot of enthusiasm, I opened up some of the previous exercises and timed retraining them just to find out that the performance (on the awesome GPU) was more than 4 times slower... It was running on the GPU there, but it had only about 10% utilization, after some research I found out the builds for Windows are poor and trying to optimize it and build from sources was the last thing I wanted to do in Windows... So I came back to my machine, fixed my issues, made sure TensorFlow was capable of running on my GPU (GeForce GTX 1050 mobile, 2GB), timed it again and once more to find out that is was slooooow - about 2 times slower than just on my CPU (i5-7300HQ CPU @ 2.50GHz with 8GB RAM). I purged the whole TensorFlow, carefully went through its installation from source documentation, configured and built with optimization for my hardware. Now the CPU's performance has increased by 15% and I am getting almost exactly 2 times slower performance on the GPU compared to the CPU... The GPU utilization is 15% - 20%, according to forums it just might be that the models/inputs are just too small to have high utilization - other than that I have no idea why it is so slow...

### Importing

In [ ]:
# for testing whether CPU or GPU is faster on a problem
NO_GPU = False

if NO_GPU:
    import os
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    
import numpy as np
import matplotlib.pyplot as plt
import keras

### Model related functions

In [ ]:
def build_simple_model(neurons, activations, input_shape, verbose=True,
               optimizer='sgd', loss='categorical_crossentropy', metrics=['acc']):
    model = keras.models.Sequential()
    zzz = zip(neurons, activations)
    
    n, a = next(zzz)
    model.add(keras.layers.Dense(n, input_shape=input_shape, activation=a))
    for n, a in zzz:
        model.add(keras.layers.Dense(n, activation=a))
        
    if verbose:
        print("Model summary:")
        model.summary()
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

In [ ]:
def build_model(*layers, verbose=False,
               optimizer='adam', loss='categorical_crossentropy', metrics=['acc'],
               compile_kwargs={}):
    
    model = keras.models.Sequential()
    
    for layer in layers:
        model.add(layer)
        
    if verbose:
        print("Model summary:")
        model.summary()
    
    for kw in ('optimizer', 'loss', 'metrics'):
        if not kw in compile_kwargs:
            compile_kwargs[kw] = locals()[kw]
    model.compile(**compile_kwargs)
    
    return model

In [ ]:
model.fit(**kwargs)

### Displaying

In [ ]:
def plot_history(history, figsize=(15,4), title='', columns=2, start_x_from=0):
    """Graphs a history for each key (combines validation and training keys into one plot).
    
    start_x_from=N skips the first N entries.
    
    History can be a whole training history class or just a dict."""
    
    if hasattr(history, 'history'): # full history given
        history = history.history   # only that history is enough
        
    assert hasattr(history, 'keys')
    keys = [key for key in history.keys() if not key.startswith("val_")]
    assert keys # there is one at least
    epochs = list(range(1,len(history[keys[0]])+1)) # all should have the same size list
    
    rows = np.ceil(len(keys)/columns).astype('int')
    
    fig=plt.figure(figsize=figsize)
    f = plt.title(title)
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)
    
    i = 1
    for key in sorted(keys):
        valkey = "val_" + key
        fig.add_subplot(rows, columns, i)
        i += 1
        plt.plot(epochs[start_x_from:], history[key][start_x_from:], label="Training " + key,
                 marker='.', color='#00A287', linestyle='')
        
        late_avg = np.mean(history[key][(len(history[key]) * 90) // 100 : ])
        plt.plot((epochs[start_x_from], epochs[-1]), (late_avg, late_avg),
                 color="#74E600", label='Mean {:.3f}'.format(late_avg))
        if valkey in history:
            plt.plot(epochs[start_x_from:], history[valkey][start_x_from:], label='Validation ' + key,
                    marker='+', color='#DF004F', linestyle='')
            
            late_avg = np.mean(history[valkey][(len(history[valkey]) * 90) // 100 : ])
            plt.plot((epochs[start_x_from], epochs[-1]), (late_avg, late_avg),
                     color="#FF6700", label='Mean {:.3f}'.format(late_avg))
        plt.legend()
        
    plt.show()

In [ ]:
def show_imgs(images, columns=9, figsize=(15,7), title=''):
    """Displays images in a grid"""
    
    fig=plt.figure(figsize=figsize)
    f = plt.title(title)
    
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)
    
    rows = np.ceil(len(images)/columns).astype('int')
    
    for i in range(1, len(images)+1):
        fig.add_subplot(rows, columns, i)
        f = plt.imshow(images[i-1], cmap=plt.cm.binary)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)
        
    plt.show()

In [ ]:
def show_vals(img):
    """Shows values in a 2D normalized grayscale image."""
    for r in img:
        for c in r:
            if c:
                print('{: 2.0f}'.format(c*100), end='', sep='')
            else:
                print('  ', end='', sep='')
        print()

### Dataset related functions

In [ ]:
def all_same_length(*args):
    """Returns True if all arguments have equal len(.)"""
    return all(len(a) == len(args[0]) for a in args)

def shuffle_together(*numpy_arrays):
    """Shuffles numpy arrays in unison, returns a tuple.
    
    (applies the same random permutation to all of them,
    so they have to be the same length on axis=0)"""
    
    assert all_same_length(*numpy_arrays)
    permut = np.random.permutation(len(numpy_arrays[0]))
    return tuple(a[permut] for a in numpy_arrays)

def normalize(shift, scale, *args):
    """For each arg returns: (arg + shift) * scale"""
    return tuple((arg + shift) * scale for arg in args)

### K-fold related functions

In [ ]:
def get_inner_slice_from_k_split(k, ith_slice, numpy_array):
    """returns the validation part of the data"""
    split_length = len(numpy_array) // k 
    return numpy_array[ith_slice * split_length : (ith_slice + 1) * split_length]
    
def get_outer_slice_from_k_split(k, ith_slice, numpy_array):
    """returns the training part of the data"""
    split_length = len(numpy_array) // k 
    return np.concatenate((
        numpy_array[:ith_slice * split_length],
        numpy_array[(ith_slice + 1) * split_length:]),
        axis=0)

def get_k_fold_sets(k, ith_set, x_data, y_targets):
    """returns the i-th training/validation set in k-fold,
    returns as: (x_train, y_train), (x_valid, y_valid)"""
    
    x_train = get_outer_slice_from_k_split(k, ith_set, x_data)
    y_train = get_outer_slice_from_k_split(k, ith_set, y_targets)
    
    x_valid = get_inner_slice_from_k_split(k, ith_set, x_data)
    y_valid = get_inner_slice_from_k_split(k, ith_set, y_targets)
    
    return (x_train, y_train), (x_valid, y_valid)

def kfoldize(K, train_data, train_targets,
             epochs, batch_size,
             model_factory,
             verbose_level=1):
    """Runs K-fold validation on the supplied dataset.
    Returns the training histories for all K trainings as a list.
    
    model_factory must be callable and return the desired model (fresh copy)
    
    Any non-false verbose_level informs upon begining new training,
    verbose_level >= 10 sets trainings to be verbose."""
    
    histories = []
    for i in range(K):
    
        if verbose_level:
            print('fold', i+1, 'out of', K)
    
        kfold_training, kfold_validation = get_k_fold_sets(K, i, train_data, train_targets)
        model = model_factory()
        hist = model.fit(*kfold_training, validation_data=kfold_validation,
                        epochs=epochs, batch_size=batch_size, verbose=(verbose_level >= 10))
        histories.append(hist.history)
        
    return histories

def average_histories(histories, for_keys=None):
    """Averages the results per key,
    if for_keys==None, all keys are used,
    otherwise a key in for_keys must be
    containded in each history"""
    
    if for_keys == None:
        for_keys = histories[0].keys()
        
    average_history = {}
    for key in for_keys:
        average_history[key] = [
            np.mean([hist[key][epoch] for hist in histories]) for epoch in range(len(histories[0][key]))]
        
    return average_history